In [2]:
from collections import OrderedDict
import torch

In [3]:
ckpt=torch.load('resnet18-f37072fd.pth')

In [12]:
import numpy as np

In [207]:
ckpt['bn1.running_mean'].shape

torch.Size([64])

In [208]:
ckpt['bn1.running_var'].shape

torch.Size([64])

In [209]:
ckpt['bn1.weight'].shape

torch.Size([64])

In [210]:
ckpt['bn1.bias'].shape

torch.Size([64])

In [211]:
from torchvision.models import resnet18

In [240]:
m=resnet18(pretrained=True)

/home/lyan/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lyan/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/lyan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:04<00:00, 9.51MB/s]


In [241]:
s=list(m.children())

In [264]:
def write_graph(model, output_name):
    submodules=list(model.children())
    
    f = open(output_name, 'wb')
    n=3
    
    header = np.zeros(256, dtype=np.int32)
    header[0]=42
    header[1]=1
    header[2] = n

    f.write(header.tobytes())
    
    for i in range(n):
        l=submodules[i]
            
        if type(l) == torch.nn.Conv2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=1 # type for conv
            header[1]=l.in_channels; header[2]=l.out_channels
            header[3]=l.kernel_size[0]; header[4]=l.kernel_size[1]
            header[5]=l.stride[0];header[6]=l.stride[1]
            
            header[7]=l.padding[0];header[8]=l.padding[1]
            if l.bias is None:
                header[9]=0
            else:
                header[9]=int(l.bias)
            header[10]=l.groups
            header[11]=l.dilation[0];header[12]=l.dilation[1];
            
            weight=l.weight.detach().to(torch.float32).numpy()
            for i in range(len(weight.shape)):
                header[13+i] = weight.shape[i]

            f.write(header.tobytes())
            f.write(weight.tobytes())
            if l.bias is not None or l.bias:
                f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
            
        elif type(l) == torch.nn.BatchNorm2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=2 # type for conv
            header[1] = l.num_features
            
            f.write(header.tobytes())
            f.write(l.weight.detach().to(torch.float32).numpy().tobytes())
            f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
            f.write(l.running_mean.detach().to(torch.float32).numpy().tobytes())
            f.write(l.running_var.detach().to(torch.float32).numpy().tobytes())
        elif type(l) == torch.nn.ReLU:
            header = np.zeros(256, dtype=np.int32)
            header[0]=3 # type for conv
            f.write(header.tobytes())
            
    f.close()

In [265]:
write_graph(m, 'r18.bin')

In [128]:
def write_data(x, name, output_name):
    with open(output_name, 'wb') as f:
        header = np.zeros(256, dtype=np.int32)
        header[0]=42
        header[1]=1
        for i in range(len(x.shape)):
            header[2+i] = x.shape[i]
        
        f.write(header.tobytes())
        
        f.write(x.tobytes())

In [157]:
import cv2

In [200]:
img=cv2.imread('/home/lyan/Downloads/000000062491.jpg')
# img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img=cv2.resize(img, (64,64))
img=torch.tensor(img).reshape(1,3,64,64).to(torch.float32) / 255

In [35]:
import matplotlib.pyplot as plt

In [170]:
write_data(img.numpy(), '', 'test_img.bin')
write_data(ckpt['conv1.weight'].detach().to(torch.float32).numpy(), 'conv1.weight', 'test.bin')

In [202]:
y=torch.nn.functional.conv2d(img, ckpt['conv1.weight'].detach() , )

In [204]:
y[0,0].reshape(-1)[0:20]

tensor([-1.1196, -0.9304, -0.6296, -0.4332, -0.5840, -1.0241, -1.4404, -1.6431,
        -1.7220, -1.7734, -1.8642, -1.9655, -1.9549, -1.8048, -1.5887, -1.3734,
        -1.1993, -1.0883, -1.0240, -0.9661])